# Introduction

Ask a home buyer to describe their dream house, and they probably won’t begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition’s dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.
With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.
The potential for creative feature engineering provides a rich opportunity for fun and learning. This dataset lends itself to advanced regression techniques like random forests and gradient boosting with the popular XGBoost library.

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques


# Data Loading & Preparation
## Load up Python libraries

In [ ]:
import numpy as np
import pandas as pd 
from pandas.api.types import CategoricalDtype
import math
import re
import warnings
from pathlib import Path
import pprint
from IPython.display import HTML, display

# Plotting stuff
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Stats and ML libraries
import scipy.stats as stats
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
import optuna
import lightgbm as lgb

# Pandas display formatting - to show more columns
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500


## Read data from csv

In [ ]:
## load training data
# na_filter - Pandas impute NA as missing, but here it means either "No" for cateoricals or missing for numerics, so we have to handle this manually
house_train = pd.read_csv(Path('data/train.csv'), na_filter=False)
house_test = pd.read_csv(Path('data/test.csv'), na_filter=False)

## Understand the data

In [ ]:
print(f"Training shape: {house_train.shape}")
print(f"Testing shape: {house_test.shape}")

In [ ]:
house_train.info()

In [ ]:
house_train.head()

In [ ]:
house_train.describe()

## Data Types
Pandas guesses the types of all the columns when it scans a csv. It does a reasonable job most of the time, but we need to check carefully.

### Build a categorical data reference

In [ ]:
# Derive categorical features and all their possible categories from data_description.txt
categorical_lookup = {}
with open(Path('data/data_description.txt'), "r", encoding="utf-8") as f:
    lines = f.readlines()

    for line in lines:
        if (not re.search(r"^\s", line)) & (": " in line):
            # if line doesnt start with space/tab, and contains :, its a key
            col, _ = line.strip().split(":")
            categorical_lookup[col] = []
        else:
            # else if its longer than zero it's a value
            if len(line.strip()) > 0:
                val = line.strip().split("\t")[0].strip()
                categorical_lookup[col].append(val)
                
# Print some formatted results
categorical_lookup = {k:categorical_lookup[k] for k in list(categorical_lookup.keys()) if len(categorical_lookup[k]) > 0}
for key in categorical_lookup.keys():
    print(f"{key}: {categorical_lookup[key][:5]}")

Looks like thse contain ordinal and nominal variables as well some that could be converted to numerics (OverallQual, OverallCond).

Other columns contain values missing from the lookup (MSSubClass, MSZoning, Neighborhood, BldgType, Exterior2nd, SaleType).

We need to look at these individually to understand them.

### Fixing Categories and names

In [ ]:
# Check that the values in the category lookup match those in the data
def test_missing_categories():
    missing_categories = {}
    for col in categorical_lookup.keys():
        cat_diff = list(set(house_train[col].dropna().unique()) - set(categorical_lookup[col]))
        
        # level_diff = list(set([str(i) for i in house_train[varname] if str(i) != "nan"]) - set(categorical_lookup[varname]))
        if len(cat_diff) > 0:
            missing_categories[col] = cat_diff
    return(missing_categories)

pprint.pprint(test_missing_categories())

Above we saw that certain of the values that occur in the data set, are missing from the categorical lookup, lets fix them.

We can automate a lot of things, but some things need to be done manually, especially during EDA.

In [ ]:
# The remainder of the categorical errors appear to be typos in the lookup, so lets update the lookup
# small helper to replace values in the category lookup
def cat_val_replacer(colname, old, new):
    try:
        idx = categorical_lookup[colname].index(old)
        categorical_lookup[colname][idx] = new
    except:
        pass

In [ ]:
# MSZoning
print(test_missing_categories()["MSZoning"])
# print(house_train["MSZoning"].unique())
# print(house_test["MSZoning"].unique())
print(categorical_lookup["MSZoning"])

In [ ]:
cat_val_replacer("MSZoning", 'C', 'C (all)')
print(categorical_lookup["MSZoning"])

In [ ]:
# Neighborhood
print(test_missing_categories()["Neighborhood"])
# print(house_train["Neighborhood"].unique())
# print(house_test["Neighborhood"].unique())
print(categorical_lookup["Neighborhood"])

In [ ]:
cat_val_replacer("Neighborhood", 'Names', 'NAmes')
print(categorical_lookup["Neighborhood"])

In [ ]:
# BldgType
print(test_missing_categories()["BldgType"])
print(house_train["BldgType"].unique())
print(house_test["BldgType"].unique())
print(categorical_lookup["BldgType"])

In [ ]:
cat_val_replacer("BldgType", '2FmCon', '2fmCon')
cat_val_replacer("BldgType", 'Duplx', 'Duplex')
cat_val_replacer("BldgType", 'TwnhsI', 'Twnhs')
print(categorical_lookup["Neighborhood"])

In [ ]:
# Exterior2nd
print(test_missing_categories()["Exterior2nd"])
# print(house_train["Exterior2nd"].unique())
# print(house_test["Exterior2nd"].unique())
print(categorical_lookup["Exterior2nd"])

In [ ]:
cat_val_replacer("Exterior2nd", 'WdShing', 'Wd Shng')
cat_val_replacer("Exterior2nd", 'CemntBd', 'CmentBd')
cat_val_replacer("Exterior2nd", 'BrkComm', 'Brk Cmn')
print(categorical_lookup["Exterior2nd"])

In [ ]:
# MSSubClass
print(test_missing_categories()["MSSubClass"])
print(categorical_lookup["MSSubClass"])

In [ ]:
# MSSubClass needs to be converted to string - why not numeric?
house_train["MSSubClass"] = house_train["MSSubClass"].astype(str)

In [ ]:
# Check again
test_missing_categories()

In [ ]:
# the NA values in Electrical and MasVnrType are different, they represent missing data
# replace these values in the data set.
house_train["MasVnrType"] = house_train["MasVnrType"].replace('NA', None)
house_test["MasVnrType"] = house_test["MasVnrType"].replace('NA', None)
house_train["Electrical"] = house_train["Electrical"].replace('NA', None)
house_test["Electrical"] = house_test["Electrical"].replace('NA', None)

OverallCond and OverallQual look like they should be left as numerics since theyre numeric ordinal, with equal spacing, so drop them from the category lookup

In [ ]:
del categorical_lookup["OverallCond"]
del categorical_lookup["OverallQual"]

In [ ]:
test_missing_categories()

A couple of last things

In [ ]:
# Convert the bool column to 1/0
house_train["CentralAir"] = house_train["CentralAir"].apply(lambda x: 1 if x == "Y" else 0)
house_test["CentralAir"] = house_test["CentralAir"].apply(lambda x: 1 if x == "Y" else 0)
del categorical_lookup["CentralAir"]
# house_train["CentralAir"]

## Split categorical into ordinal & nominal

In [ ]:
# Take another look at our categoricals...
for key in categorical_lookup.keys():
    print(f"{key}: {categorical_lookup[key][:5]}")

Looking through these, it looks like ordinas would be the following variables, with their categories ordered as follows. These sorts of things you need to do manually...

In [ ]:
# manual build ordinal lookup
ordinal_lu = {
    "LotShape": ['Reg', 'IR1', 'IR2', 'IR3'],
    "Utilities": ['ELO', 'NoSeWa', 'NoSewr', 'AllPub'],
    "LandSlope": ['Gtl', 'Mod', 'Sev'],
    "ExterQual": ['Po','Fa', 'TA', 'Gd', 'Ex'],
    "ExterCond": ['Po','Fa', 'TA', 'Gd', 'Ex'],
    "BsmtQual": ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    "BsmtCond": ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    "BsmtExposure": ['NA', 'No', 'Mn', 'Av', 'Gd'],
    "BsmtFinType1": ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    "BsmtFinType2": ['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
    "HeatingQC": ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    "KitchenQual": ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
    "Functional": ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
    "FireplaceQu": ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    "GarageFinish": ['NA', 'Unf', 'RFn', 'Fin'],
    "GarageQual": ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    "GarageCond": ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
    "PavedDrive": ['N', 'P', 'Y'],
    "PoolQC": ['NA', 'Fa', 'TA', 'Gd', 'Ex']
}

non_nom = list(ordinal_lu.keys())
nominal_lu = {k: categorical_lookup[k] for k in categorical_lookup if k not in non_nom}

non_num = list(ordinal_lu.keys()) + list(nominal_lu.keys()) + ["Id"]
numerics = [c for c in house_train.columns if c not in non_num]

print("nominals")
for key in nominal_lu.keys():
    print(f"{key}: {nominal_lu[key][:5]}")

print("\nordinals")
for key in ordinal_lu.keys():
    print(f"{key}: {ordinal_lu[key][:5]}")

print("\nnumerics")
print(numerics)

## Converting column data types
convert the data types:
- Categorical to ordinal, nominal or boolean
- Make sure the rest are numeric

In [ ]:
def column_typer(pdf:pd.DataFrame):
    for colname in pdf.columns:
        if colname in ordinal_lu.keys():
            cat_def = CategoricalDtype(categories=ordinal_lu[colname], ordered=True)
            pdf[colname] = pdf[colname].astype(str).astype(cat_def)

        elif colname in nominal_lu.keys():
            cat_def = CategoricalDtype(categories=nominal_lu[colname], ordered=False)
            pdf[colname] = pdf[colname].astype(str).astype(cat_def)

        elif colname in numerics:
            # coerce will convert "NA" to Nan
            pdf[colname] = pd.to_numeric(pdf[colname], errors='coerce')

    pdf["Id"] = pdf["Id"].astype(int)
    return(pdf)

house_train = column_typer(house_train.copy())
house_test = column_typer(house_test.copy())

## Saving data
Its a good idea to save your data from time to time, so that you can more easily pick up where you left off

In [ ]:
house_train.to_pickle(Path("house_train_tidy.pkl"))
house_test.to_pickle(Path("house_test_tidy.pkl"))

In [ ]:
house_train = pd.read_pickle(Path("house_train_tidy.pkl"))
house_test = pd.read_pickle(Path("house_test_tidy.pkl"))

# Visualizations
## Distributions
### SalePrice Probability Plot
Probability plot, a graphical method for comparing sample data against the quantiles of a specified theoretical distribution.

In [ ]:
stats.probplot(house_train['SalePrice'], dist='norm', plot=plt);

In [ ]:
stats.probplot(np.log(house_train['SalePrice']), dist='norm', plot=plt);

### Histograms

In [ ]:
# SalePrice
# Take a look at the distribution of the target variable
fig,ax = plt.subplots(1,2, sharex=False)
house_train["SalePrice"].plot(kind="hist", ax=ax[0], color='tab:blue', title="SalePrice", bins=20)
np.log(house_train["SalePrice"]).plot(kind="hist", ax=ax[1], color='tab:orange', title="log(SalePrice)", bins=20);

In [ ]:
# Look at distributions of all numeric data
house_train[numerics].hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

## Understand features in terms of SalePrice

In [ ]:
def facet_plotter(pdf:pd.DataFrame, x_vars:list, y_var:str, col_wrap:int=4):
    g = sns.FacetGrid(pd.DataFrame(x_vars), col=0, col_wrap=col_wrap, sharex=False)
    for ax, x_var in zip(g.axes, x_vars):
        sns.boxplot(data=pdf, x=x_var, y=y_var, ax=ax)
    g.tight_layout()

In [ ]:
# Look at Ordinals in terms of sames price
facet_plotter(house_train, ordinal_lu.keys(), "SalePrice")

In [ ]:
# Nominals
facet_plotter(house_train, nominal_lu.keys(), "SalePrice")

In [ ]:
# Numeric variables
f = pd.melt(house_train, id_vars=['SalePrice'], value_vars=numerics)
sns.lmplot(
    data=f, x="value", y='SalePrice', col="variable", lowess=True, 
    col_wrap=4, height=3, aspect=1,scatter_kws={'alpha':0.3, "linewidth":0}, line_kws={"color": "C1"}, facet_kws={"sharex": False});

## Correlations
### Correlations between Continuous Variables

In [ ]:
corrmat = house_train[numerics].corr(method='spearman')
plt.subplots(figsize=(10, 8))
sns.heatmap(corrmat, annot=False, cmap="YlGnBu", square=True);

### Correlations between Ordinal Variables

In [ ]:
# Create a temporary datframe containing the ordinal order values
pdf_tmp = house_train[ordinal_lu.keys()].copy()
for c in ordinal_lu.keys():
    pdf_tmp[c] = pdf_tmp[c].cat.codes
corrmat = pdf_tmp.corr(method='kendall')
plt.subplots(figsize=(10, 8))
sns.heatmap(corrmat, annot=False, cmap="YlGnBu", square=True);

## Correlations with SalePrice
### Ordinals with SalePrice
Spearmans Rank Correlation, good for continuous and ordinal, robust to outliers, but **assumes linearity**

In [ ]:
# function to calculate correlations between features and SalePrice
def spearman_categoricals(pdf, features):
    spr = pd.DataFrame()
    spr['feature'] = features
    spr['spearman'] = [pdf[f].cat.codes.corr(pdf['SalePrice'], 'spearman') for f in features]
    spr = spr.sort_values('spearman')
    plt.figure(figsize=(6, 0.25*len(features)))
    sns.barplot(data=spr, y='feature', x='spearman', orient='h')
spearman_categoricals(house_train, ordinal_lu.keys())

### Numerics with SalePrice

In [ ]:
# function to calculate correlations between features and SalePrice
def spearman_numerics(pdf, features):
    spr = pd.DataFrame()
    spr['feature'] = features
    spr['spearman'] = [pdf[f].corr(pdf['SalePrice'], 'spearman') for f in features]
    spr = spr.sort_values('spearman')
    plt.figure(figsize=(6, 0.25*len(features)))
    sns.barplot(data=spr, y='feature', x='spearman', orient='h')
spearman_numerics(house_train, numerics)

# Pre-Processing
## Understanding missingness

In [ ]:
house_train = pd.read_pickle(Path("house_train_tidy.pkl"))
house_test = pd.read_pickle(Path("house_test_tidy.pkl"))

In [ ]:
def missing_plotter(pdf):
    total = pdf.isnull().sum().sort_values(ascending=False)
    percent = (pdf.isnull().sum()/pdf.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, round(percent*100,2)], axis=1, keys=['Total', 'Percent'])
    missing_data = missing_data[missing_data.Total>=1]
    if len(missing_data) > 0:
        missing_data.plot.bar(subplots=True); # Pandas
    else:
        print("No missing data")
    return(missing_data)
missing_plotter(house_train)

In [ ]:
missing_plotter(house_test)[:5]

## Dealing with some missing data
GarageYrBlt - numeric, we will impute that a little later on

## LotFrontage

In [ ]:
pdf=house_train.copy()
pdf["frontage"] = pdf["LotFrontage"].apply(lambda x: 1 if x>0 else x)
pdf.groupby(["LotConfig", "frontage"], dropna=False).size()

missing data across all config sizes. We will impute this below

## Missing Pattern of GarageYrBlt and some feature engineering

In [ ]:
# How does GarageYrBlt relate to YearBuilt?
plt_data = house_train[["YearBuilt", "GarageYrBlt"]].copy()
plt_data["GarageYrBlt_nafill"] = plt_data["GarageYrBlt"].fillna(plt_data["YearBuilt"])
plt_data["missing"] = plt_data["GarageYrBlt"].apply(lambda x: 1 if pd.isnull(x) else 0)

sns.scatterplot(data=plt_data, x="YearBuilt", y="GarageYrBlt_nafill", hue="missing");
# Looks like, in most cases, garages are built when the houses are built

### Transforming GarageYrBlt

- What does missing data here mean? likely that there is no garage
- We will try and capture the information in GarageYrBlt in 2 other variables
    - **hasGarage**: indicates whether there is a garage or not
    - **GarageBlt**: years + 1 after construction that the garage was built. 
        - 0 indicates no garage, + integer indicates garage built after years + 1

In [ ]:
## Create new feature: hasGarage
## Impute GarageYrBlt with YearBuilt

def garage_feat(pdf:pd.DataFrame) -> pd.DataFrame:
    # hasGarage: 0 or 1
    pdf["hasGarage"] = [0 if pd.isnull(x) else 1 for x in pdf["GarageYrBlt"]]

    # GarageBlt
    pdf["GarageBlt"] = pdf["GarageYrBlt"] - pdf["YearBuilt"] + 1
    pdf.loc[pdf["GarageYrBlt"].isna(), "GarageBlt"] = 0
    pdf = pdf.drop(columns=["GarageYrBlt"])
    return(pdf)

house_train = garage_feat(house_train.copy())
house_test = garage_feat(house_test.copy())


In [ ]:
sns.scatterplot(data=house_train, x="YearBuilt", y="GarageBlt", hue="hasGarage");

## Imputing other missing data using KNN and Mode

In [ ]:
from sklearn.impute import KNNImputer, SimpleImputer

# KNN imputer for numeric values
imputer_num = KNNImputer(n_neighbors=5)

# Simple mode imputer for categoricals
# imputer_cat = SimpleImputer(strategy="most_frequent") # dont use as it drops catagorical info, roll your own
def mode_imputer(pdf):
    modes = pdf.mode().T
    for f in pdf.columns:
        pdf[f] = pdf[f].fillna(modes.loc[f][0])
    return(pdf)


In [ ]:
def imputer(pdf:pd.DataFrame):
    # Columns have changed since we first identified all the types, so re-ID numeric vs categorical features
    numeric_cols = pdf.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = [c for c in pdf.columns if c not in numeric_cols]

    # Impute missing values separately
    pdf_num_imputed = pd.DataFrame(imputer_num.fit_transform(pdf[numeric_cols]), columns=numeric_cols)
    pdf_cat_imputed = mode_imputer(pdf[categorical_cols].copy())

    # Join the imputed versions back together
    pdf_concat = pd.concat([pdf_num_imputed, pdf_cat_imputed], axis=1)
    pdf_concat["Id"] = pdf_concat["Id"].astype(int)
    return(pdf_concat)

In [ ]:
house_train = imputer(house_train)
house_test = imputer(house_test)

In [ ]:
# Look at missing again
display(missing_plotter(house_train)[:5])
display(missing_plotter(house_test)[:5])

## Low Variance Predictors

In [ ]:
def nearZeroVar(pdf, features, freqCut=95/5, uniqueCut=10):
    features = list(features)

    n_unique = []
    freqRatio = []
    percentUnique = []
    for f in features:
        tmp = pdf.groupby(f).size()
        tmp = tmp[tmp>0].sort_values(ascending=False).reset_index(drop=True)
        n_unique.append(len(tmp))
        freqRatio.append(tmp[0]/tmp[1])
        percentUnique.append((len(tmp)/len(pdf[f]))*100)

    res = pd.DataFrame()
    res['feature'] = features
    res['freqRatio'] = freqRatio
    res['percentUnique'] = percentUnique

    res["zeroVar"] = [True if i == 1 else False for i in n_unique]
    res["nzv"] = (res['freqRatio'] > freqCut) & (res['percentUnique'] < uniqueCut)

    return(res)

In [ ]:
nzv = nearZeroVar(house_train, house_train.columns, freqCut=99/1, uniqueCut=5)
nzv = nzv[nzv["nzv"]]
nzv

In [ ]:
# Drop these
house_train = house_train.drop(columns=nzv["feature"])

## More Feature engineering: Transforming YearRemodAdd
- Remodel date (same as construction date if no remodeling or additions)

In [ ]:
plt_data = house_train[["YearBuilt", "YearRemodAdd"]].copy()
plt_data["remodel==constr"] = plt_data["YearRemodAdd"]==plt_data["YearBuilt"]
plt_data["remodel==constr"]

sns.scatterplot(data=plt_data, x="YearBuilt", y="YearRemodAdd", hue="remodel==constr");

### Create new features from YearRemodAdd
- it looks like many houses built before 1950 was remodelled in 1950 - unlikely, lets assume that if a house was built before 1950, and remodelled in 1950, in fact it was never remodelled and someone simply used 1950 as a default settign for all those houses.

- We will try and capture the information in YearRemodAdd in 2 other variables
    - **isRemod**: indicates whether the house is remodelled
    - **RemodAdd**: years + 1 after construction that the house was remodelled. 
        - 0 indicates no remodel, + integer indicates remodelled after years + 1

In [ ]:
pdf = house_train.copy()
pdf.loc[pdf["YearRemodAdd"] == 1950, "YearRemodAdd"] = pdf.loc[pdf["YearRemodAdd"] == 1950, "YearBuilt"]
sns.scatterplot(data=pdf, x="YearBuilt", y="YearRemodAdd");


In [ ]:
def remod_feat(pdf:pd.DataFrame) -> pd.DataFrame:
    # Correct remodelling date for 1950 remodelled
    pdf.loc[pdf["YearRemodAdd"] == 1950, "YearRemodAdd"] = pdf.loc[pdf["YearRemodAdd"] == 1950, "YearBuilt"]
    pdf["isRemod"] = (pdf["YearRemodAdd"]==pdf["YearBuilt"]).astype(int)

    pdf["RemodAdd"] = pdf["YearRemodAdd"] - pdf["YearBuilt"]
    pdf = pdf.drop(columns=["YearRemodAdd"])
    return(pdf)

house_train = remod_feat(house_train.copy())
house_test = remod_feat(house_test.copy())

In [ ]:
sns.scatterplot(data=house_train, x="YearBuilt", y="RemodAdd", hue="isRemod");

## Transforming SalePrice to log scale
Transforms can sometimes help models converge

In [ ]:
house_train["logSalePrice"] = np.log(house_train['SalePrice'])

## Save data

In [ ]:
house_train.to_pickle(Path("house_train_preproc.pkl"))
house_test.to_pickle(Path("house_test_preproc.pkl"))

In [ ]:
house_train = pd.read_pickle(Path("house_train_preproc.pkl"))
house_test = pd.read_pickle(Path("house_test_preproc.pkl"))

# Model Training and Parameter Tuning
We will use Microsofts LightGBM framework to build a regression tree model for our data, and then see if we can improve performance via some hyper parameter tuning.
## Prepare the dataset for modelling

In [ ]:
# ID features and target variables
features = [c for c in house_train.columns if c not in ["Id", "SalePrice", "logSalePrice"]]
target1 = "SalePrice"
target2 = "logSalePrice"

# split into train and validation sets
X, y = house_train[features], house_train[target1]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## Train a vanilla model to set a baseline

In [ ]:
# Quieten some LGBM warnings
warnings.filterwarnings("ignore", category=UserWarning)

# SKLearn API
params = { 
    'boosting': 'gbdt',
    'objective': 'poisson',
    'metric': 'rmse',
    'verbose': -1
}

evals={}
model = lgb.LGBMRegressor(**params)
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=-1,
    callbacks=[
        lgb.record_evaluation(evals), 
        lgb.early_stopping(100),
        lgb.log_evaluation(0)
    ], 
)

lgb.plot_metric(evals);

y_pred = model.predict(X_valid)
rmse_vanilla = mean_squared_error(y_valid, y_pred, squared=False)
print("\nRoot Mean Squared Error:", rmse_vanilla)

In [ ]:
# Predicted vs Observed
plt.scatter(y_valid, y_pred, alpha=.3, linewidth=0)
plt.plot([0,max(y_valid)], [0, max(y_valid)], color="red")
plt.xlabel("Observed")
plt.ylabel("Predicted")
plt.title("Predicted vs Observed")
plt.show()

## Feature importance
Lets find out whether we can perhaps ignore some of our features, building simpler better models

In [ ]:
def feat_imp_plotter(m, topn=0):
    feature_imp = pd.DataFrame({'importance':m.feature_importances_}, index=m.feature_name_)\
        .sort_values("importance")
    if topn > 0:
        feature_imp = feature_imp.tail(topn)
        height = math.ceil(topn/5)
    else:
        feature_imp = feature_imp[feature_imp["importance"] > 0]
        height = int(len(feature_imp)/5)
    feature_imp = feature_imp[feature_imp["importance"] > 0]
    tot = feature_imp["importance"].sum()
    feature_imp["cumulative_importance_percent"] = ((feature_imp['importance'][::-1]/tot)*100).cumsum()
    feature_imp.plot.barh(subplots=True, sharex=False, sharey=True, layout=(1,2), figsize=(10,height))
    return(feature_imp)
    

In [ ]:
feature_imp = feat_imp_plotter(model)

### 5.3.1 Test model performance with fewer metrics
Lets get a general idea what happens if we drop some less important features... This can be useful when you have many thousands of features, and need a simpler faster model.

Generally it's at the expense of some score-performance, but that is ok sometimes.

In [ ]:
# Small helper function to quickly train and test the impact of feature changes on model performance
def model_feat_test(feature_imp, cutoff):
    # ID features and target variables
    features = list(feature_imp.loc[feature_imp["cumulative_importance_percent"]<=cutoff].index)
    target = "SalePrice"

    # split into train and validation sets
    X, y = house_train[features], house_train[target]
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    # SKLearn API
    params = { 
        'boosting': 'gbdt',
        'objective': 'poisson',
        'metric': 'rmse',
        'verbose': -1
    }

    model = lgb.LGBMRegressor(**params)    
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        verbose=-1,
        callbacks=[
            lgb.early_stopping(100),
            lgb.log_evaluation(0)
        ], 
    )

    y_pred = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_pred, squared=False)
    
    return(rmse)


In [ ]:
# display(feature_imp.tail())

cutoffs = [85, 90, 95, 100]
results = pd.DataFrame({'rmse':np.nan}, index=cutoffs)
# results.loc[85]["rmse"] = 5
# results
for cutoff in cutoffs:
    results.loc[cutoff]["rmse"] = model_feat_test(feature_imp, cutoff)
    # results.append(model_feat_test(feature_imp, cutoff))

In [ ]:
display(results)
rmse_feat = results['rmse'].min()
print(f"To beat: {rmse_vanilla}")
print(f"best: {rmse_feat}")

Trimming off features which contribute to our model has an impact on score as expected, but trimming off zero contributing features actually improved the score a bit, so lets do that.

### Retain only contributing features

In [ ]:
features = list(feature_imp.index)
target1 = "SalePrice"
target2 = "logSalePrice"

# Split the features from the target variable
X, y = house_train[features], house_train[target1]

## 5.4 Hyperparameter tuning and Cross validation
Hyperparameter tuning: testing various combinations of model parameters, looking for optima

Cross validation: measuring model parameter performance across subsets of your data to measure performance and importantly stability

### Use Optuna to manage out hyperparameter tuning

In [ ]:
# Build OpTunas objective function
def objective(trial, X, y):
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)

    #static parameters
    param_static = { 
        'boosting_type': 'gbdt',
        'objective': 'poisson',
        'metric': 'rmse',
        'verbose': -1,
        'n_estimators': 10000
    }
    
    # the parameter grid from where unique parameters will be drawn
    param_grid = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, step=0.001),
        "num_leaves": trial.suggest_int("num_leaves", 20, 60, step=1),
        "max_depth": trial.suggest_int("max_depth", 3, 50)
    }

    # set up cross validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    # for storing scores from the K-Folds
    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgb.LGBMRegressor(
            **param_static, 
            **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric="rmse",
        categorical_feature="auto",
            verbose=-1,
            callbacks=[
                lgb.early_stopping(100, verbose=0),
                lgb.log_evaluation(0)
            ]
        )
        y_pred = model.predict(X_test)
        cv_scores[idx] = mean_squared_error(y_test, y_pred, squared=False)

    return np.mean(cv_scores)
    # return(0)

Run the grid search

In [ ]:
# Suppress optuna training logs
optuna.logging.set_verbosity(optuna.logging.WARNING)
# optuna.logging.set_verbosity(optuna.logging.INFO)

# Create an OpTuna study
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")

# Call the Optuna Objective function, and begine some simple hyperparameter tuning
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20, show_progress_bar=True)

In [ ]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

**Optuna** has a few useful canned plots that can show us interesting info about interactions between our hyperparameters.

https://optuna.readthedocs.io/en/stable/reference/visualization/index.html


In [ ]:
# Plot parameter relations as scatter plots with colors indicating ranks of target value.
fig = optuna.visualization.plot_rank(study, target_name="rmse")
fig.show()

### Train the final model using optimal parameters

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_static = { 
    'boosting_type': 'gbdt',
    'objective': 'poisson',
    'metric': 'rmse',
    'verbose': -1,
    'n_estimators': 10000
}

evals={}
model = lgb.LGBMRegressor(
    **param_static, 
    **study.best_params)
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=-1,
    callbacks=[
        lgb.record_evaluation(evals), 
        lgb.early_stopping(100),
        lgb.log_evaluation(0)
    ], 
)

lgb.plot_metric(evals);

y_pred = model.predict(X_test)
rmse_final = mean_squared_error(y_test, y_pred, squared=False)
print(f"\nTo beat: {rmse_feat}")
print("Root Mean Squared Error:", rmse_final)

Some quick hyperparameter tuning has improves out RMSE score on the validation set.